In [ ]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
sys.path.append(os.getcwd()+'/../resort/rasotools-master/')
import rasotools
np.seterr(all='warn')
np.errstate(divide='raise')

## Calculating the estimate time for calculation:

In [ ]:
files = glob.glob('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/*.nc')
files[6020]

In [ ]:
b = os.path.getsize(files[6020])
b

In [ ]:
t = 2633.3687977790833/60
t

In [ ]:
b/t

Speed: ~2.8 MB/(min thread)

In [ ]:
dirsize = 0 
for i in files:
    dirsize += os.path.getsize(i)
dirsize

In [ ]:
mintime = (dirsize)/(b/t)

In [ ]:
mintime/60/24 , 'days'

~ 1 week on 20 cores -> more cores available on the jet?

---
---

In [ ]:
filesout = glob.glob('/raid60/scratch/uli/converted/*')
filesout

In [ ]:
# with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc') as test:
with eua.CDMDataset(filesout[-1]) as test:

    print(test)
    print(test.recordindices)
    print(test.observations_table)
    print(test.era5fb)
    print(test.era5fb['an_depar@body'][:])
    print(test.era5fb['fg_depar@body'][:])
    a = test.to_dataframe(groups=['observations_table', 'era5fb'], variables=['z_coordinate', 'date_time','observed_variable', 'observation_value', 'conversion_flag', 'conversion_method', 'fg_depar@body', 'an_depar@body','biascorr@body', 'biascorr_fg@body'])
a

In [ ]:
b = a[a.conversion_method > 0]
b = b.reset_index(drop=True)
print(b.observed_variable.drop_duplicates())
b

In [ ]:
b = a[a.conversion_method > 0]
b = b[np.isnan(b['fg_depar@body'])]
b = b.reset_index(drop=True)
print(b.observed_variable.drop_duplicates())
b

In [ ]:
b = a[a.conversion_method > 0]
b = b[~np.isnan(b['fg_depar@body'])]
b = b.reset_index(drop=True)
print(b.observed_variable.drop_duplicates())
b